In [1]:
#!pip install georasters
#!pip install geoindex
#!pip install spark
#!pip install pyspark

In [1]:
import pandas as pd
import georasters as gr
import matplotlib.pyplot as plt
from geoindex import GeoGridIndex, GeoPoint
import os
from tqdm import tqdm, tqdm_pandas

In [2]:
pd.set_option('display.max_columns', 100)

## Load Water Data

In [4]:
os.getcwd()

'/Users/alexjamar/Documents/Berkeley/Capstone/Capstone_Water_Access/Population'

In [3]:
water_data = pd.read_csv("/Users/alexjamar/Documents/Berkeley/Capstone/Capstone_Water_Access//data/Water_Point_Data_Exchange_Complete_Dataset.csv")

/Users/alexjamar/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (2,5,7,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
water_data.head()

,Row ID,#country_name,#water_source,#water_tech,#status_id,#management,#pay,#installer,#install_year,#status,#source,#adm1,#adm2,#wpdx_id,#report_date,#country_id,#activity_id,#data_lnk,#orig_lnk,photo_lnk,#converted,#created,#updated,#lat_deg,#lon_deg,Location,Count,#fecal_coliform_presence,#fecal_coliform_value,#subjective_quality
0,1,"Tanzania, United Republic of",Spring,Gravity Communal standpipe,no,NaN,Never pay,RWE,1972.0,Status:Not functional|Quantity:Dry|Quality:Soft,SNV,Arusha,Longido,wpdx-00000001,01/05/2008 12:00:00 AM +0000,TZ,NaN,http://www.dropbox.com/s/50779gal9v4z4y6/TZ%20...,NaN,NaN,"""#water_tech, #country_id, #data_lnk""",10/23/2015 01:03:07 AM +0000,10/23/2015 01:03:07 AM +0000,-2.83257,36.3090,"(-2.83257, 36.309)",1,NaN,NaN,NaN
1,2,"Tanzania, United Republic of",Rainwater harvesting,Gravity Communal standpipe,yes,NaN,Never pay,World Vision,2005.0,Status:Functional|Breakdown Year:2007|Reason N...,SNV,Arusha,Longido,wpdx-00000002,01/01/2008 12:00:00 AM +0000,TZ,NaN,http://www.dropbox.com/s/50779gal9v4z4y6/TZ%20...,NaN,NaN,"""#water_tech, #country_id, #data_lnk""",10/23/2015 01:03:07 AM +0000,10/23/2015 01:03:07 AM +0000,-2.84781,36.5284,"(-2.84781, 36.5284)",1,NaN,NaN,NaN
2,3,"Tanzania, United Republic of",Rainwater harvesting,Gravity Communal standpipe,yes,NaN,Never pay,World Vision,2005.0,Status:Functional|Quantity:Seasonal|Quality:Soft,SNV,Arusha,Longido,wpdx-00000003,01/01/2008 12:00:00 AM +0000,TZ,NaN,http://www.dropbox.com/s/50779gal9v4z4y6/TZ%20...,NaN,NaN,"""#water_tech, #country_id, #data_lnk""",10/23/2015 01:03:07 AM +0000,10/23/2015 01:03:07 AM +0000,-2.84796,36.5288,"(-2.84796, 36.5288)",1,NaN,NaN,NaN
3,4,"Tanzania, United Republic of",Rainwater harvesting,Gravity Communal standpipe,yes,NaN,Never pay,World Vision,2005.0,Status:Functional|Quantity:Seasonal|Quality:Soft,SNV,Arusha,Longido,wpdx-00000004,01/01/2008 12:00:00 AM +0000,TZ,NaN,http://www.dropbox.com/s/50779gal9v4z4y6/TZ%20...,NaN,NaN,"""#water_tech, #country_id, #data_lnk""",10/23/2015 01:03:07 AM +0000,10/23/2015 01:03:07 AM +0000,-2.85318,36.5276,"(-2.85318, 36.5276)",1,NaN,NaN,NaN
4,5,Nicaragua,NaN,NaN,yes,NaN,NaN,NaN,2011.0,NaN,El Porvenier,Matagalpa,Terrabona,wpdx-00000005,01/01/2011 12:00:00 AM +0000,NI,NaN,http://www.dropbox.com/s/0f70mln5roxhy29/ProjT...,https://www.waterpointdata.org/dataset/resourc...,NaN,"#data_lnk, #lon_deg, #lat_deg, #country_id",04/08/2016 08:35:49 AM +0000,04/08/2016 08:35:49 AM +0000,12.74070,-86.0121,"(12.7407, -86.0121)",1,NaN,NaN,NaN


In [11]:
#Create df with only South Sudan water
s_sudan_water = water_data[water_data['#country_name'] == 'South Sudan']

In [12]:
s_sudan_water.head()

,Row ID,#country_name,#water_source,#water_tech,#status_id,#management,#pay,#installer,#install_year,#status,#source,#adm1,#adm2,#wpdx_id,#report_date,#country_id,#activity_id,#data_lnk,#orig_lnk,photo_lnk,#converted,#created,#updated,#lat_deg,#lon_deg,Location,Count,#fecal_coliform_presence,#fecal_coliform_value,#subjective_quality
230318,386946,South Sudan,Borehole,,yes,NaN,NaN,NaN,NaN,NaN,Water Information Management System (WIMS) dev...,NaN,NaN,wpdx-00381053,09/14/2006 12:00:00 AM +0000,SS,14,NaN,https://data.hdx.rwlabs.org/dataset/south-suda...,NaN,"#orig_lnk, #country_id, #report_date",01/19/2016 08:21:27 PM +0000,01/19/2016 08:21:27 PM +0000,4.86168,31.5562,"(4.86168, 31.5562)",1,NaN,NaN,NaN
230475,386947,South Sudan,Borehole,,yes,NaN,NaN,NaN,NaN,NaN,Water Information Management System (WIMS) dev...,NaN,NaN,wpdx-00381040,07/09/2006 12:00:00 AM +0000,SS,15,NaN,https://data.hdx.rwlabs.org/dataset/south-suda...,NaN,"#orig_lnk, #country_id, #report_date",01/19/2016 08:21:27 PM +0000,01/19/2016 08:21:27 PM +0000,4.84128,31.5680,"(4.84128, 31.568)",1,NaN,NaN,NaN
230571,386948,South Sudan,Borehole,,yes,NaN,NaN,NaN,NaN,NaN,Water Information Management System (WIMS) dev...,NaN,NaN,wpdx-00381038,06/24/2006 12:00:00 AM +0000,SS,16,NaN,https://data.hdx.rwlabs.org/dataset/south-suda...,NaN,"#orig_lnk, #country_id, #report_date",01/19/2016 08:21:27 PM +0000,01/19/2016 08:21:27 PM +0000,4.81970,31.5528,"(4.8197, 31.5528)",1,NaN,NaN,NaN
230988,386949,South Sudan,Borehole,,yes,NaN,NaN,NaN,NaN,NaN,Water Information Management System (WIMS) dev...,NaN,NaN,wpdx-00381039,06/30/2006 12:00:00 AM +0000,SS,17,NaN,https://data.hdx.rwlabs.org/dataset/south-suda...,NaN,"#orig_lnk, #country_id, #report_date",01/19/2016 08:21:27 PM +0000,01/19/2016 08:21:27 PM +0000,4.83258,31.5704,"(4.83258, 31.5704)",1,NaN,NaN,NaN
231242,386950,South Sudan,Borehole,,yes,NaN,NaN,NaN,NaN,NaN,Water Information Management System (WIMS) dev...,NaN,NaN,wpdx-00381032,06/21/2006 12:00:00 AM +0000,SS,18,NaN,https://data.hdx.rwlabs.org/dataset/south-suda...,NaN,"#orig_lnk, #country_id, #report_date",01/19/2016 08:21:27 PM +0000,01/19/2016 08:21:27 PM +0000,4.86613,31.5021,"(4.86613, 31.5021)",1,NaN,NaN,NaN


## Radius-based Search

In [13]:
s_sudan_pop = pd.read_csv("South Sudan_pop_data.csv")

In [14]:
#Create Geo Index of South Sudan Population Data
geo_index = GeoGridIndex()
for index, row in s_sudan_pop.iterrows():
    geo_index.add_point(GeoPoint(row['y'], row['x'], ref=row['value']))

In [15]:
#Create Geo Index of South Sudan Water Point Data
geo_index_water = GeoGridIndex()
for index, row in s_sudan_water.iterrows():
    try:
        geo_index_water.add_point(GeoPoint(row['#lat_deg'], row['#lon_deg'], ref=row['#status_id']))
    except:
        print("Row ID:", row['Row ID'])
        print("Lat:", row['#lat_deg'])
        print("Long:", row['#lon_deg'])

In [16]:
#Calculate population within x distance of water point
def calculate_population_within_x_km(row, index, km):
    center_point = GeoPoint(row['#lat_deg'], row['#lon_deg'])
    total_population = 0
    try:
        for point,distance in index.get_nearest_points(center_point, km, 'km'):
            total_population += point.ref
    except:
        print("Invalid data - Record skipped")
    #print("Total population within", km, "kilometers:", int(total_population))
    return total_population

In [17]:
#Calculate # of water points within x distance of water point
def calculate_water_points_within_x_km(row, index, km):
    center_point = GeoPoint(row['#lat_deg'], row['#lon_deg'])
    total_water_points = 0
    total_functioning_water_points = 0
    try:
        for point,distance in index.get_nearest_points(center_point, km, 'km'):
            total_water_points += 1
            if point.ref == 'yes':
                total_functioning_water_points += 1
    except:
        print("Invalid data - Record skipped")
        
    #Adjust for self
    total_water_points -= 1
    if row['#status_id'] == 'yes':
        total_functioning_water_points -= 1
    return (total_water_points, total_functioning_water_points)

In [18]:
#Add 1 km radius population data to each well
tqdm.pandas()
s_sudan_water['1 km population'] = s_sudan_water.progress_apply(lambda row: calculate_population_within_x_km(row, geo_index, 1),
                                                  axis = 1)

100%|██████████| 6088/6088 [7:23:53<00:00,  4.37s/it]  
/Users/alexjamar/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [19]:
#Add 1 km radius water point data to each water point
tqdm.pandas()
s_sudan_water['1 km water points (total, functioning)'] = s_sudan_water.progress_apply(lambda row: calculate_water_points_within_x_km(row, geo_index_water, 1),
                                                  axis = 1)

100%|██████████| 6088/6088 [00:17<00:00, 343.37it/s]
/Users/alexjamar/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [20]:
s_sudan_water.head(2)

,Row ID,#country_name,#water_source,#water_tech,#status_id,#management,#pay,#installer,#install_year,#status,#source,#adm1,#adm2,#wpdx_id,#report_date,#country_id,#activity_id,#data_lnk,#orig_lnk,photo_lnk,#converted,#created,#updated,#lat_deg,#lon_deg,Location,Count,#fecal_coliform_presence,#fecal_coliform_value,#subjective_quality,1 km population,"1 km water points (total, functioning)"
230318,386946,South Sudan,Borehole,,yes,NaN,NaN,NaN,NaN,NaN,Water Information Management System (WIMS) dev...,NaN,NaN,wpdx-00381053,09/14/2006 12:00:00 AM +0000,SS,14,NaN,https://data.hdx.rwlabs.org/dataset/south-suda...,NaN,"#orig_lnk, #country_id, #report_date",01/19/2016 08:21:27 PM +0000,01/19/2016 08:21:27 PM +0000,4.86168,31.5562,"(4.86168, 31.5562)",1,NaN,NaN,NaN,6168.487,"(3, 1)"
230475,386947,South Sudan,Borehole,,yes,NaN,NaN,NaN,NaN,NaN,Water Information Management System (WIMS) dev...,NaN,NaN,wpdx-00381040,07/09/2006 12:00:00 AM +0000,SS,15,NaN,https://data.hdx.rwlabs.org/dataset/south-suda...,NaN,"#orig_lnk, #country_id, #report_date",01/19/2016 08:21:27 PM +0000,01/19/2016 08:21:27 PM +0000,4.84128,31.5680,"(4.84128, 31.568)",1,NaN,NaN,NaN,5079.374,"(1, 1)"


In [21]:
s_sudan_water['1 km population'] = s_sudan_water['1 km population'].apply(lambda x: int(round(x,0)))

/Users/alexjamar/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [22]:
s_sudan_water['1 km total water points'] = s_sudan_water['1 km water points (total, functioning)'].apply(lambda x: str(x).split(',')[0][1:])

/Users/alexjamar/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [23]:
s_sudan_water['1 km functioning water points'] = s_sudan_water['1 km water points (total, functioning)'].apply(lambda x: str(x).split(',')[1][:-1])

/Users/alexjamar/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [24]:
s_sudan_water.head()

,Row ID,#country_name,#water_source,#water_tech,#status_id,#management,#pay,#installer,#install_year,#status,#source,#adm1,#adm2,#wpdx_id,#report_date,#country_id,#activity_id,#data_lnk,#orig_lnk,photo_lnk,#converted,#created,#updated,#lat_deg,#lon_deg,Location,Count,#fecal_coliform_presence,#fecal_coliform_value,#subjective_quality,1 km population,"1 km water points (total, functioning)",1 km total water points,1 km functioning water points
230318,386946,South Sudan,Borehole,,yes,NaN,NaN,NaN,NaN,NaN,Water Information Management System (WIMS) dev...,NaN,NaN,wpdx-00381053,09/14/2006 12:00:00 AM +0000,SS,14,NaN,https://data.hdx.rwlabs.org/dataset/south-suda...,NaN,"#orig_lnk, #country_id, #report_date",01/19/2016 08:21:27 PM +0000,01/19/2016 08:21:27 PM +0000,4.86168,31.5562,"(4.86168, 31.5562)",1,NaN,NaN,NaN,6168,"(3, 1)",3,1
230475,386947,South Sudan,Borehole,,yes,NaN,NaN,NaN,NaN,NaN,Water Information Management System (WIMS) dev...,NaN,NaN,wpdx-00381040,07/09/2006 12:00:00 AM +0000,SS,15,NaN,https://data.hdx.rwlabs.org/dataset/south-suda...,NaN,"#orig_lnk, #country_id, #report_date",01/19/2016 08:21:27 PM +0000,01/19/2016 08:21:27 PM +0000,4.84128,31.5680,"(4.84128, 31.568)",1,NaN,NaN,NaN,5079,"(1, 1)",1,1
230571,386948,South Sudan,Borehole,,yes,NaN,NaN,NaN,NaN,NaN,Water Information Management System (WIMS) dev...,NaN,NaN,wpdx-00381038,06/24/2006 12:00:00 AM +0000,SS,16,NaN,https://data.hdx.rwlabs.org/dataset/south-suda...,NaN,"#orig_lnk, #country_id, #report_date",01/19/2016 08:21:27 PM +0000,01/19/2016 08:21:27 PM +0000,4.81970,31.5528,"(4.8197, 31.5528)",1,NaN,NaN,NaN,134,"(0, 0)",0,0
230988,386949,South Sudan,Borehole,,yes,NaN,NaN,NaN,NaN,NaN,Water Information Management System (WIMS) dev...,NaN,NaN,wpdx-00381039,06/30/2006 12:00:00 AM +0000,SS,17,NaN,https://data.hdx.rwlabs.org/dataset/south-suda...,NaN,"#orig_lnk, #country_id, #report_date",01/19/2016 08:21:27 PM +0000,01/19/2016 08:21:27 PM +0000,4.83258,31.5704,"(4.83258, 31.5704)",1,NaN,NaN,NaN,2049,"(0, 0)",0,0
231242,386950,South Sudan,Borehole,,yes,NaN,NaN,NaN,NaN,NaN,Water Information Management System (WIMS) dev...,NaN,NaN,wpdx-00381032,06/21/2006 12:00:00 AM +0000,SS,18,NaN,https://data.hdx.rwlabs.org/dataset/south-suda...,NaN,"#orig_lnk, #country_id, #report_date",01/19/2016 08:21:27 PM +0000,01/19/2016 08:21:27 PM +0000,4.86613,31.5021,"(4.86613, 31.5021)",1,NaN,NaN,NaN,2265,"(2, 2)",2,2


In [25]:
s_sudan_abridged = s_sudan_water[['Row ID', '#country_name', '#status_id', '#lat_deg', '#lon_deg', '1 km population',
                        '1 km total water points', '1 km functioning water points']]

In [26]:
s_sudan_abridged.head()

,Row ID,#country_name,#status_id,#lat_deg,#lon_deg,1 km population,1 km total water points,1 km functioning water points
230318,386946,South Sudan,yes,4.86168,31.5562,6168,3,1
230475,386947,South Sudan,yes,4.84128,31.5680,5079,1,1
230571,386948,South Sudan,yes,4.81970,31.5528,134,0,0
230988,386949,South Sudan,yes,4.83258,31.5704,2049,0,0
231242,386950,South Sudan,yes,4.86613,31.5021,2265,2,2


In [28]:
s_sudan_abridged.to_csv("Abridged_South_Sudan_Water_and_Population_Data.csv", index = False)